In [1]:
%load_ext autoreload
%autoreload 2

import sys
import os
import dotenv
from pathlib import Path

env_file = "../.env"

if os.path.exists(env_file):
    dotenv.load_dotenv(env_file, verbose=True)
    print("Loaded environment variables from .env file.")

cwd = os.getcwd()
# for some reason appending to PATH you need it to be string
sys.path.append(str(Path(cwd).parent / "src"))
hf_access_token = os.getenv("HUGGINGFACE_API_KEY")

Loaded environment variables from .env file.


In [2]:
import torch
from research_tools import get_gpus_available
from transformers import AutoModelForCausalLM, AutoTokenizer
from transformers import LlamaForCausalLM, LlamaTokenizer


os.environ["CUDA_VISIBLE_DEVICES"] = ",".join([str(i) for i in get_gpus_available()])
model_dtype = torch.bfloat16
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
assert device.type == "cuda", "No GPU available."

model_name = "meta-llama/Meta-Llama-3-8B"

model: LlamaForCausalLM = AutoModelForCausalLM.from_pretrained(
    model_name, token=hf_access_token, torch_dtype=model_dtype
)
model = model.to(device)

tokenizer: LlamaTokenizer = AutoTokenizer.from_pretrained(
    model_name, token=hf_access_token
)
tokenizer.pad_token_id = tokenizer.eos_token_id
tokenizer.padding_side = "left"

Loading checkpoint shards:   0%|          | 0/4 [00:00<?, ?it/s]

In [4]:
from peft import get_peft_model, LoraConfig

lora_rank = 64
lora_alpha = 8

lora_config = LoraConfig(
    r=lora_rank,
    lora_alpha=lora_alpha,
    target_modules=["q_proj", "v_proj"],
)

model = get_peft_model(model, lora_config)

In [5]:
from unlearn_order.dataset import load_dataset

data_dir = Path("../data/random_bd")

splits = list(range(10))
n_train = 1
n_val = 1

train_files = [f"split_{splits[i]}.jsonl" for i in range(n_train)]
val_files = [f"split_{splits[i]}.jsonl" for i in range(n_train, n_train + n_val)]
combined_files = train_files + val_files

train_dataset = load_dataset(data_dir, train_files)
val_dataset = load_dataset(data_dir, val_files)
combined_dataset = load_dataset(data_dir, combined_files)

In [ ]:
# original experiment
# learn T, V
# forget T, V
# learn T
from unlearn_order.finetune import finetune_model
from unlearn_order.eval import eval_dataset

tolerance = 0.05
batch_size = 32

model, loss_traj, acc_traj = finetune_model(
    model,
    tokenizer,
    combined_dataset,
    batch_size=batch_size,
    tolerance=tolerance
)
print(f"Combine train acc: {acc_traj[-1]}")
model, loss_traj, acc_traj = finetune_model(
    model,
    tokenizer,
    combined_dataset,
    batch_size=batch_size, 
    shuffle_labels=True,
    tolerance=tolerance
)
t_acc = eval_dataset(model, tokenizer, train_dataset, batch_size=batch_size)
v_acc = eval_dataset(model, tokenizer, val_dataset, batch_size=batch_size)
print(f"Unlearn train accuracy: {t_acc}")
print(f"Unlearn val accuracy: {v_acc}")
model, loss_traj, acc_traj = finetune_model(
    model,
    tokenizer,
    train_dataset,
    batch_size=batch_size,
    tolerance=tolerance
)

t_acc = eval_dataset(model, tokenizer, train_dataset, batch_size=batch_size)
v_acc = eval_dataset(model, tokenizer, val_dataset, batch_size=batch_size)
print(f"Train accuracy: {t_acc}")
print(f"Val accuracy: {v_acc}")


  1%|          | 1/100 [00:07<12:56,  7.84s/it]

Epoch 0 loss: 0.07362737454426517 acc: 0.22929936305732485


 11%|█         | 11/100 [01:21<10:58,  7.40s/it]

Epoch 10 loss: 0.03097087590937402 acc: 0.29936305732484075


 21%|██        | 21/100 [02:35<09:45,  7.41s/it]

...

  1%|          | 1/100 [00:07<11:46,  7.14s/it]

Epoch 0 loss: 0.0011517352551404627 acc: 0.9012738853503185


 11%|█         | 11/100 [01:18<10:37,  7.16s/it]

Epoch 10 loss: 0.0006057851960634884 acc: 0.9012738853503185


 19%|█▉        | 19/100 [02:15<09:39,  7.15s/it]

In [ ]:
from unlearn_order.pipeline import run_pipeline

batch_size = 28
tolerance = 0.05
lr = 3e-6

run_pipeline(
    model,
    tokenizer,
    [("f", "combined", combined_dataset), ("u", "unlearn", combined_dataset), 
     ("e", "eval_train", train_dataset), ("e", "eval_val", val_dataset)
     ("f", "retrain_train", train_dataset),
     ("e", "eval_train", train_dataset), ("e", "eval_val", val_dataset)
     ],
    batch_size=batch_size,
    tolerance=tolerance,
    lr=lr
)


In [ ]:
from unlearn_order.pipeline import run_pipeline

batch_size = 28
tolerance = 0.05

run_pipeline(
    model,
    tokenizer,
    [("f", "train_train", train_dataset), 
     ("f", "val_train", val_dataset),
     ("e", "eval_train", train_dataset), ("e", "eval_val", val_dataset)

     ("u", "unlearn", combined_dataset), 
     ("e", "eval_train", train_dataset), ("e", "eval_val", val_dataset)
     
     ("f", "retrain_train", train_dataset),
     ("e", "eval_train", train_dataset), ("e", "eval_val", val_dataset)
     ],
    batch_size=batch_size,
    tolerance=tolerance,
    lr=lr
)


In [ ]:
from unlearn_order.pipeline import run_pipeline

batch_size = 28
tolerance = 0.05

run_pipeline(
    model,
    tokenizer,
    [
     ("f", "val_train", val_dataset),
    ("f", "train_train", train_dataset), 
     ("e", "eval_train", train_dataset), ("e", "eval_val", val_dataset)

     ("u", "unlearn", combined_dataset), 
     ("e", "eval_train", train_dataset), ("e", "eval_val", val_dataset)
     
     ("f", "retrain_train", train_dataset),
     ("e", "eval_train", train_dataset), ("e", "eval_val", val_dataset)
     ],
    batch_size=batch_size,
    tolerance=tolerance,
    lr=lr
)
